In [299]:
import pandas as pd
import nfl_data_py as nfl

In [300]:
df = pd.read_pickle('2013-2023_plays.pkl')

In [301]:
df_fg = df[df['field_goal_attempt']==1]

In [302]:
# df_fg['play_duration'] = 
df_fg[['time','drive_game_clock_end']]

df_fg['drive_game_clock_end'] = pd.to_datetime(df_fg['drive_game_clock_end'], format='%M:%S')
df_fg['time'] = pd.to_datetime(df_fg['time'], format='%M:%S')


df_fg['play_time'] = (df_fg['time']-df_fg['drive_game_clock_end']).dt.seconds

df_fg['play_time'].mode()


/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_91158/2581555462.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg['drive_game_clock_end'] = pd.to_datetime(df_fg['drive_game_clock_end'], format='%M:%S')
/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_91158/2581555462.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg['time'] = pd.to_datetime(df_fg['time'], format='%M:%S')
/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_91158/2581555462.py:8: PerformanceWarning

0    4
Name: play_time, dtype: int32

In [303]:
class GameState:
    def __init__(self, **kwargs):
        self.receive_2h_ko = self._validate_receive_2h_ko(kwargs.get('receive_2h_ko', 0))
        self.home_team = self._validate_home_team(kwargs.get('home_team', ""))
        self.posteam = self._validate_posteam(kwargs.get('posteam', ""))
        self.score_differential = self._validate_score_differential(kwargs.get('score_differential', 0))
        self.half_seconds_remaining = self._validate_half_seconds_remaining(kwargs.get('half_seconds_remaining', 1800))
        self.game_seconds_remaining = self._validate_game_seconds_remaining(kwargs.get('game_seconds_remaining', 3600))
        self.spread_line = self._validate_spread_line(kwargs.get('spread_line', [1, 3, 4, 7, 14]))
        self.down = self._validate_down(kwargs.get('down', 1))
        self.ydstogo = self._validate_ydstogo(kwargs.get('ydstogo', 10))
        self.yardline_100 = self._validate_yardline_100(kwargs.get('yardline_100', 75))
        self.posteam_timeouts_remaining = self._validate_timeouts(kwargs.get('posteam_timeouts_remaining', 3))
        self.defteam_timeouts_remaining = self._validate_timeouts(kwargs.get('defteam_timeouts_remaining', 3))
        self.goal_to_go = self._validate_goal_to_go(kwargs.get('goal_to_go',0))

    def update_attribute(self, attribute, value):
        if hasattr(self, attribute):
            validation_method = getattr(self, f'_validate_{attribute}')
            setattr(self, attribute, validation_method(value))
        else:
            raise AttributeError(f"GameState has no attribute '{attribute}'")

    def _validate_receive_2h_ko(self, value):
        if value not in [0, 1]:
            raise ValueError("receive_2h_ko must be either 0 or 1")
        return value

    def _validate_posteam(self, value):
        if not isinstance(value, str) or len(value) < 1 or len(value) > 4:
            raise ValueError("home_team and posteam must be a string with length between 1 and 4")
        return value
    
    def _validate_home_team(self, value):
        if not isinstance(value, str) or len(value) < 1 or len(value) > 4:
            raise ValueError("home_team and posteam must be a string with length between 1 and 4")
        return value

    def _validate_score_differential(self, value):
        if not isinstance(value, int):
            raise ValueError("score_differential must be an integer")
        return value

    def _validate_game_seconds_remaining(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("half_seconds_remaining and game_seconds_remaining must be non-negative integers")
        return value

    def _validate_spread_line(self, value):
        if not isinstance(value, int):
            raise ValueError("spread_line must be an integer")
        return value

    def _validate_down(self, value):
        if not isinstance(value, int) or value < 1 or value > 4:
            raise ValueError("down must be an integer between 1 and 4")
        return value

    def _validate_ydstogo(self, value):
        if not isinstance(value, int) or value < 1 or value > 99:
            raise ValueError("ydstogo must be an integer between 1 and 99")
        return value

    def _validate_yardline_100(self, value):
        if not isinstance(value, int) or value < 1 or value > 99:
            raise ValueError("yardline_100 must be an integer between 1 and 99")
        return value

    def _validate_timeouts(self, value):
        if not isinstance(value, int) or value < 0 or value > 3:
            raise ValueError("posteam_timeouts_remaining and defteam_timeouts_remaining must be integers between 0 and 3")
        return value

    def _validate_goal_to_go (self, value):
        if value not in [0, 1]:
            raise ValueError("goal_to_go must be either 0 or 1")
        return value
    
    def _validate_half_seconds_remaining(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("half_seconds_remaining must be a non-negative integer")
        return value
    



    def __str__(self):
        return f"GameState(receive_2h_ko={self.receive_2h_ko}, home_team='{self.home_team}', " \
               f"posteam='{self.posteam}', score_differential={self.score_differential}, " \
               f"half_seconds_remaining={self.half_seconds_remaining}, " \
               f"game_seconds_remaining={self.game_seconds_remaining}, spread_line={self.spread_line}, " \
               f"down={self.down}, ydstogo={self.ydstogo}, yardline_100={self.yardline_100}, " \
               f"posteam_timeouts_remaining={self.posteam_timeouts_remaining}, " \
               f"defteam_timeouts_remaining={self.defteam_timeouts_remaining}, "\
               f"goal_to_go={self.goal_to_go})"

In [304]:
def fg_made(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    game_state.update_attribute('score_differential',-(game_state.score_differential+3))

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
        
    game_state.update_attribute('yardline_100',75)
    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

    game_state.update_attribute('goal_to_go',0)


In [305]:
def fg_miss(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)


    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
    predicted_line = 100-(game_state.yardline_100+7)
    if predicted_line > 99:
        game_state.update_attribute('yardline_100',99)
    elif predicted_line < 1:
         game_state.update_attribute('yardline_100',1)
    else:
        game_state.update_attribute('yardline_100',100-(game_state.yardline_100+7))
    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

    game_state.update_attribute('goal_to_go',0)


In [306]:
import pickle
def punt(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
    
    with open('punt_yardage_model.pkl', 'rb') as file:
        # Punt Model predicts new yard line given current yard line
        punt_model = pickle.load(file)

    with open('punt_touchback_model.pkl', 'rb') as file:
        punt_touchback_model = pickle.load(file)
    
    if punt_touchback_model.predict_proba([[game_state.yardline_100]])[0][1] > 0.5:
        game_state.update_attribute('yardline_100',80)

    else:
        game_state.update_attribute('yardline_100',round(punt_model.predict([[game_state.yardline_100]])[0]))

    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

In [307]:
def go_for_it_fail(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)

    with open('GFI_fail_model.pkl', 'rb') as file:
    # GFI Model predicts  yard gain/loss given current game state
        GFI_fail_model = pickle.load(file)

    predicted_line = 100-(game_state.yardline_100 - GFI_fail_model.predict([[game_state.yardline_100,game_state.ydstogo,game_state.goal_to_go]])[0])
    
    if predicted_line >99:
        game_state.update_attribute('yardline_100',99)
    elif predicted_line < 1:
         game_state.update_attribute('yardline_100',1)
            
    else:
        game_state.update_attribute('yardline_100',round(predicted_line))

    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)


In [308]:

def go_for_it_success(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)


    if game_state.goal_to_go ==1:
        game_state.update_attribute('score_differential',-(game_state.score_differential+7))
        # New yardage after kickoff
        game_state.update_attribute('yardline_100',75)
        game_state.update_attribute('down',1)
        game_state.update_attribute('ydstogo',10)
        game_state.update_attribute('goal_to_go',0)
    else:
        with open('GFI_success.pkl', 'rb') as file:
            # GFI Model predicts  yard gain/loss given current game state
            GFI_success_model = pickle.load(file)
        predicted_line =game_state.yardline_100 -  GFI_success_model.predict([[game_state.yardline_100,game_state.ydstogo]])[0]
        if predicted_line >= game_state.yardline_100:
            game_state.update_attribute('yardline_100',1)
        # if predicted_line <1:
        #     game_state.update_attribute('yardline_100',1)
        # if predicted_line > 99:
        #     game_state.update_attribute('yardline_100',99)
        else:
            game_state.update_attribute('yardline_100',(game_state.yardline_100 - round(predicted_line)))
        
    
        

In [309]:
import pandas as pd

def create_game_states():
    game_states_data = []
    game_states_list = [] #also generate game state class objects

    for yardline_100 in range(1, 100):
        for ydstogo in range(1, 11):
            goal_to_go = 1 if yardline_100 <= 10 else 0

            game_state_data = {
                'receive_2h_ko': 1,
                'home_team': "SEA",
                'posteam': "SEA",
                'score_differential': 0,
                'half_seconds_remaining': 900,
                'game_seconds_remaining': 2700,
                'spread_line': -3,
                'down': 4,
                'ydstogo': ydstogo,
                'yardline_100': yardline_100,
                'posteam_timeouts_remaining': 3,
                'defteam_timeouts_remaining': 3,
                'goal_to_go': goal_to_go
            }

            game_states_data.append(game_state_data)
            game_states_list.append(GameState(**game_state_data))

    game_states_df = pd.DataFrame(game_states_data)
    return game_states_df,game_states_list

In [310]:
game_states,game_state_list = create_game_states()

In [311]:
for game_state in game_states:
    print(game_state)

receive_2h_ko
home_team
posteam
score_differential
half_seconds_remaining
game_seconds_remaining
spread_line
down
ydstogo
yardline_100
posteam_timeouts_remaining
defteam_timeouts_remaining
goal_to_go


In [312]:
# game_states
game_state_list

In [313]:
import copy
test = copy.deepcopy(game_state_list[0])

print(game_state_list[0])

GameState(receive_2h_ko=1, home_team='SEA', posteam='SEA', score_differential=0, half_seconds_remaining=900, game_seconds_remaining=2700, spread_line=-3, down=4, ydstogo=1, yardline_100=1, posteam_timeouts_remaining=3, defteam_timeouts_remaining=3, goal_to_go=1)


In [314]:
print(test)

GameState(receive_2h_ko=1, home_team='SEA', posteam='SEA', score_differential=0, half_seconds_remaining=900, game_seconds_remaining=2700, spread_line=-3, down=4, ydstogo=1, yardline_100=1, posteam_timeouts_remaining=3, defteam_timeouts_remaining=3, goal_to_go=1)


In [315]:
(fg_made(test))
print(test)

GameState(receive_2h_ko=1, home_team='SEA', posteam='xxx', score_differential=-3, half_seconds_remaining=895, game_seconds_remaining=2695, spread_line=-3, down=1, ydstogo=10, yardline_100=75, posteam_timeouts_remaining=3, defteam_timeouts_remaining=3, goal_to_go=0)


In [316]:
potential_states_dict = {}

for game_state in game_state_list:
    game_state_punt = copy.deepcopy(game_state)
    game_state_FG_made = copy.deepcopy(game_state)
    game_state_FG_miss = copy.deepcopy(game_state)
    game_state_GFI_fail = copy.deepcopy(game_state)
    game_state_GFI_success = copy.deepcopy(game_state)

    punt(game_state_punt)
    fg_made(game_state_FG_made)
    fg_miss(game_state_FG_miss)
    go_for_it_fail(game_state_GFI_fail)
    go_for_it_success(game_state_GFI_success)


    potential_states = {
        "Punt": game_state_punt,
        "FG_SUCCESS": game_state_FG_made,
        "FG_FAIL": game_state_FG_miss,
        "GFI_FAIL": game_state_GFI_fail,
        "GFI_SUCCESS": game_state_GFI_success
    }
    potential_states_dict[game_state] = potential_states

/Users/ilanb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/ilanb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/ilanb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/ilanb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/ilanb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/ilanb/anaconda3/lib/

In [319]:
with open('my_dict.pkl', 'wb') as file:
    # Pickle the dictionary and write it to the file
    pickle.dump(potential_states_dict, file)

In [321]:
with open('my_class.pkl', 'wb') as file:
    pickle.dump(GameState, file)